<a href="https://colab.research.google.com/github/renhaa/faces/blob/master/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/My\ Drive/faces


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/faces


In [0]:
%run Tools.ipynb

ERROR:root:File `'Tools.ipynb.py'` not found.


In [0]:

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm, tqdm_notebook
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import adam as Adam
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import cv2
from google.colab.patches import cv2_imshow



class DCGAN():
    """ From
    https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py
    """
    def __init__(self,X,save_dir = None):
        # Input shape
        self.save_dir = save_dir
        self.X = X
        (self.n,self.img_rows,self.img_cols,self.channels) = X.shape
        # self.X = np.expand_dims(X, axis=3)
      
        # self.img_rows = X.shape[1]
        # self.img_cols = X.shape[2]
        self.dim = (self.img_rows,self.img_cols)
       
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        w,h = self.dim
        w,h = int(w/4),int(h/4)

        model = Sequential()

        model.add(Dense(128 * w * h , activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((w, h, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        #model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

      #  model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = self.X


        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        # Record losses 
        lossG = []
        lossD = []
        acc = []
        for epoch in tqdm_notebook(range(epochs)):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)


            lossG.append(g_loss)
            lossD.append(d_loss[0])
            acc.append(d_loss[1])
            # Plot the progress

            msg = f"epoch{epoch}, d_loss{d_loss[0]}, acc{d_loss[1]}, g_loss{g_loss}\n"
            with open(self.save_dir+"log.txt", "a+") as f:
                f.write(msg)

            # f = open(self.save_dir+"log", 'a') 
            # print(msg, file = f) 
            # f.close() 
            #  print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                noise = np.random.normal(0, 1, (16, self.latent_dim))
                gen_imgs = self.generator.predict(noise)
                
                # print(np.amin(gen_imgs),np.amax(gen_imgs))
                ## ReNormalize to 0..1
                # gen_imgs = gen_imgs - np.amin(gen_imgs)
                # gen_imgs = gen_imgs/np.max(gen_imgs)
                # Rescale images 0 - 1

                gen_imgs = 0.5 * gen_imgs + 0.5
                plot_images(gen_imgs, show = False, fname = f"{self.save_dir}epoch{epoch}")
                # Rescale images 0 - 1
                # gen_imgs = 0.5 * gen_imgs + 0.5
                # plot_imgs = np.zeros((16,self.img_rows,self.img_cols))
                # for i, img in enumerate(gen_imgs):
                #     plot_imgs[i,:,:] = gen_imgs[i,:,:,0]

                self.generator.save_weights(self.save_dir+"model.h5")
                np.save(self.save_dir+"training_history.npy",np.array([lossG,lossD,acc]))
                fig, axs = plt.subplots(2,1,figsize=(16,8))
                axs[0].plot(list(range(epoch + 1)),lossG,label="Discriminator loss")
                axs[0].plot(list(range(epoch + 1)),lossD,label="Generator loss")
                axs[0].set_title("Training loss")
                axs[0].set(xlabel="epoch", ylabel="loss")
                # plt.title("Training loss",size = 20)
                # plt.xlabel("epoch",size = 16)
                # plt.ylabel("loss",size = 16)
                axs[0].legend(loc="upper right")

                axs[1].plot(list(range(epoch + 1)),acc,label="Discriminator Accuracy")
                axs[1].set_title("Discriminator accuracy")
                axs[1].set(xlabel="epoch", ylabel="Accuracy")
                axs[1].legend()

                fig.savefig(self.save_dir+"loss.png")
                plt.close()


if __name__ == "__main__":

  # # FFHQ COlor
  # !rm -r results/dcgan/ffhq 
  # !mkdir results/dcgan/ffhq
  # X = np.load("data/ffhq/ffhq-1000-thumbs.npy")
  # dcgan = DCGAN(X,save_dir="results/dcgan/ffhq/")
  # dcgan.train(10001, batch_size=128, save_interval=200)

#  # FFHQ COlor 5k
#   !rm -r results/dcgan/ffhq5k 
#   !mkdir results/dcgan/ffhq5k
#   X = np.load("data/ffhq/ffhq-5000-thumbs.npy")
#   dcgan = DCGAN(X,save_dir="results/dcgan/ffhq5k/")
#   dcgan.train(50001, batch_size=128, save_interval=200)


  # Caltech gray
  # !rm -r results/dcgan/caltech_gray 
  # !mkdir results/dcgan/caltech_gray
  # X = np.load("data/caltech/caltech.npy")
  X = grayscale(X, expand_dim = True)
  dcgan = DCGAN(X,save_dir="results/dcgan/caltech_gray/")
  dcgan.train(10001, batch_size=128, save_interval=200)


Using TensorFlow backend.


NameError: ignored

In [0]:

# from PIL import Image
# img = Image.open('image.png').convert('LA')
# X = np.load("data/ffhq/ffhq-1000-thumbs.npy")
# X = np.load("data/caltech/caltech.npy")

# img = cv2.imread("data/caltech/image_0001.jpg")
# img = np.array(X[0]*255,dtype="int")
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# cv2_imshow()
# plt.imshow(gray)
# img = np.array(X[0],dtype="int")
# img = Image.fromarray(img.astype('uint8'), 'RGB').convert('LA')
# img = np.array(img)

# X = np.load("data/caltech/caltech.npy")
# X = grayscale(X, expand_dim = True)

# plot_images(X, show = True)

